In [1]:
library(ggplot2)
library(reshape2)
library(sqldf)
library(fields)
library(gridExtra)
library(ggtern)
library(clue)
library(geometry)
library(gtable)
library(goseq)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite
Loading required package: spam
Loading required package: grid
Spam version 1.4-0 (2016-08-29) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: ‘spam’

The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

Loading required package: maps
--
Consider donating at: http://ggtern.com
Even small amounts (say $10-50) are very much appreciated!
Remember to cite, run citation(package = 'ggtern') for further info.
--

Attaching package: ‘ggtern’

The following objects are masked from ‘package:gridExtra’:

    arrangeGrob, grid.arrange

The following objects are masked from ‘package:ggplot2’:

    %+%, aes, annotate, calc_element, ggplot, ggplot_build,
    ggplot_gtable, ggplot

In [2]:
folder<-"/Users/ramirezr/Dropbox/JIC/expVIPMetadatas/RefSeq1.0/TablesForExploration"

loadGeneInformation<-function(dir="../TablesForExploration"){
    path<-paste0(dir,"/CanonicalTranscript.rds")
    canonicalTranscripts<-readRDS(path)
    canonicalTranscripts$intron_length<- canonicalTranscripts$mrna_length -  canonicalTranscripts$exon_length
    canonicalTranscripts$chr_group <- substr(canonicalTranscripts$Chr,4,4)
    canonicalTranscripts$genome    <- substr(canonicalTranscripts$Chr,5,5)
    
    path<-paste0(dir, "/MeanTpms.rds")
    meanTpms <- readRDS(path)
    expressed_genes<-unique(meanTpms$gene)
    canonicalTranscripts<-canonicalTranscripts[canonicalTranscripts$Gene %in% expressed_genes, ]
    
    
    path<-paste0(dir,"/TriadMovement.rds")
    triadMovement<-readRDS(path)
    
    path<-paste0(dir,"/Triads.rds")
    triads<-readRDS(path)
    
    path<-paste0(dir,"/universe_table.csv")
    gene_universe<-read.csv(path)
    
    path<-paste0(dir, "/OntologiesForGenes.rds")
    ontologies<-readRDS(path)
    
    path<-paste0(dir, "/id_names_merged.txt")
    id_names <- read.csv(path, header=F, sep = "\t")
    
    path<-paste0(dir, "/WGCNA_table.csv")
    WGCNA <-  read.csv(path)
    
    list(canonicalTranscripts=canonicalTranscripts, 
         meanTpms=meanTpms,
         triads=triads, 
         triadMovement=triadMovement,
         gene_universe=gene_universe, 
         ontologies=ontologies,
         id_names=id_names,
         WGCNA=WGCNA
        )
}
geneInformation<-loadGeneInformation()

In [4]:
head(geneInformation$meanTpms)

value,factor,gene,samples,subset,min_mean_tpm
35.3429709,all,TraesCS1A01G000100,128,spike,0.5
778.7933277,all,TraesCS1A01G000100LC,128,spike,0.5
150.7084593,all,TraesCS1A01G000200,128,spike,0.5
0.2557844,all,TraesCS1A01G000200LC,128,spike,0.5
18.7230705,all,TraesCS1A01G000300,128,spike,0.5
3.2775778,all,TraesCS1A01G000300LC,128,spike,0.5


In [19]:
geneInformation$meanTpms$expressed<-ifelse(geneInformation$meanTpms$value > 0.5,1,0)

In [20]:
head(geneInformation$meanTpms)

value,factor,gene,samples,subset,min_mean_tpm,expressed
35.3429709,all,TraesCS1A01G000100,128,spike,0.5,1
778.7933277,all,TraesCS1A01G000100LC,128,spike,0.5,1
150.7084593,all,TraesCS1A01G000200,128,spike,0.5,1
0.2557844,all,TraesCS1A01G000200LC,128,spike,0.5,0
18.7230705,all,TraesCS1A01G000300,128,spike,0.5,1
3.2775778,all,TraesCS1A01G000300LC,128,spike,0.5,1


In [8]:
subsets<-unique(geneInformation$meanTpms$subset)

In [25]:
for(s in subsets){
    print(s)
    filename<-paste0("./Tables/ExpressedInTissue/ExpressedMinTPM05_",s,".txt")
    test<-geneInformation$meanTpms[geneInformation$meanTpms$subset==s,]
    casted<-dcast(test, gene ~ factor, value.var="expressed")
    write.table(casted, file=filename, row.names=F, sep="\t")
}

[1] "spike"
[1] "root"
[1] "leaf"
[1] "grain"
[1] "disease"
[1] "abiotic"
[1] "CS_NB_inc_stress"
[1] "CS_no_stress"
[1] "850_samples"
[1] "Development"
